# Data Science From Scratch Notes

## Chapter 4: Linear Algebra

**Linear Algebra:** The study of *vector spaces*. Linear algebra is the basis (*heh*) for many data science concepts and techniques.

## Vectors

**Vectors:** Abstractly, they are objects that can be added together to from new vectors and that can be multiplied by *scalars*, also to form new vectors. Concretely, vectors are points in some finite-dimensional space.

Vectors are a useful way to represent numeric data.

For example, if we have height, weight, and age data on people, we can treat our data as three-dimensional vectors `[height, weight, age]`.

A from-scratch representation of vectors is as lists of numbers. A list of three numbers corresponds to a vector in three-dimensional space, and vice versa. We'll accomplish this with a type alias that says a `Vector` is just a `list` of `floats`:

In [2]:
from typing import List

Vector = List[float]

height_weight_age = [70, # inches
                    170, # pounds
                    40] # years

grades = [95, # exam1
         80, # exam2
         75, # exam3
         62] # exam4

Because Python `lists` aren't vectors, we'll need to build these arithmetic tools ourselves.

### Vector Addition

First, we'll want to be able to add two vectors. Vectors add *componentwise*. If two vectors `v` and `w` are the same length, their sum is just the vector whose first element is `v[0] + w[0]`, whose second element is `v[1] + w[1]`, and so on. Thus addinge vectors `[1, 2]` and `[2, 1]` results in `[1 + 2, 2 + 1]` or `[3, 3]`.

We can implement this by `zip`-ing the vectors together and using a list comprehension to add the corresponding elements:

In [3]:
def add(v: Vector, w: Vector) -> Vector: 
    """Adds corresponding elements"""
    assert len(v) == len(w), "vectors must be the same length"
    return [v_i + w_i for v_i, w_i in zip(v, w)]

assert add([1, 2, 3], [4, 5, 6]) == [5, 7, 9]

#### Subtracting Vectors

Similarly, to subtract two vectors we just subtract the corresponding elements:

In [5]:
def subtract(v: Vector, w: Vector) -> Vector:
    """Subtracts corresponding elements"""
    assert len(v) == len(w), "vectors must be the same length"
    return [v_i - w_i for v_i, w_i in zip(v, w)]

assert subtract([5, 7, 9], [4, 5, 6]) == [1, 2, 3]

#### Componentwise sum

We'll also want to componentwise sum a list of vectors (create new vector whose first element is the sum of all the first elements, whose second element is the sum of all the second elements, and so on):

In [8]:
def vector_sum(vectors: List[Vector]) -> Vector:
    """Sums all corresponding elements"""
    # Check that vectors is not empty
    assert vectors, "no vectors provided!"
    
    # Check the vectors are all the same size
    num_elements = len(vectors[0])
    assert all(len(v) == num_elements for v in vectors), "different sizes!"
    
    # the i-th element of the result is the sum of every vector[i]
    return [sum(vector[i] for vector in vectors)
           for i in range(num_elements)]

assert vector_sum([[1, 2], [3, 4], [5, 6], [7, 8]]) == [16, 20]

### Vector Multiplication

#### Scalar Multiplication

We multiply a vector by a scalar simply by multiplying each element of the vector by that number:

In [9]:
def scalar_multiply(c: float, v: Vector) -> Vector:
    """Multiplies every element by c"""
    return [c * v_i for v_i in v]

assert scalar_multiply(2, [1, 2, 3]) == [2, 4, 6]

#### Dot Product

The dot product of two vectors is the sum of their componentwise products:

In [11]:
def dot(v: Vector, w: Vector) -> float:
    """Computes v_1 * w_1 + ... + v_n * w_n"""
    assert len(v) == len(w), "vectors must be same length"
    
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

assert dot([1, 2, 3], [4, 5, 6]) == 32 # 1 * 4 + 2 * 5 + 3 * 6

It is the length of the vector you'd get if you *projected* `v` onto `w`.

### Sum of Squares

Using the dot product, it is easy to compute a vector's *sum of squares*:

In [12]:
def sum_of_squares(v: Vector) -> float:
    """Returns v_1 * v_1 + ... + v_n * v_n"""
    return dot(v, v)

assert sum_of_squares([1, 2, 3]) == 14 # 1 * 1 + 2 * 2 + 3 * 3

### Magnitude

The sum of squares can be used to calculate the vector's *magnitude*:

In [13]:
import math

def magnitude(v: Vector) -> float:
    """Returns the magnitude of v"""
    return math.sqrt(sum_of_squares(v)) # square root function

assert magnitude([3, 4]) == 5

### Distance

Now we have all the pieces needed to compute the distance between two vectors defined as:

$$\sqrt{(v_1 - w_1)^2 + \ldots + (v_n - w_n)^2}$$

In [18]:
def squared_distance(v: Vector, w: Vector) -> float:
    """Computes (v_1 - w_1) ** 2 + ... + (v_n - w_n) ** 2"""
    return sum_of_squares(subtract(v, w))

def distance(v: Vector, w: Vector) -> float:
    """Computes the distance between v and w"""
    return math.sqrt(squared_distance(v, w))

This is possibly clearer if we write it as (the equivalent):

In [19]:
def distance(v: Vector, w: Vector) -> float:
    return magnitude(subtract(v, w))

## Matrices

A *matrix* is a two-dimensional collection of numbers. We will represent matrices as lists of lists, with each inner list having the same size and representing a *row* of the matrix. If `A` is a matrix, then `A[i][j]` is the element in the *i*th row and the *j*th column:

In [20]:
# Another type alias
Matrix = List[List[float]]

A = [[1, 2, 3], # A has 2 rows and 3 columns
     [4, 5, 6]]

B = [[1, 2], # B has 3 rows and 2 columns
     [3, 4],
     [5, 6]]

Thus, the matrix `A` has `len(A)` rows and `len(A[0])` columns, which we consider its `shape`:

In [21]:
from typing import Tuple

def shape(A: Matrix) -> Tuple[int, int]:
    """Returns (# of rows of A, # of columns of A)"""
    num_rows = len(A)
    num_cols = len(A[0]) if A else 0 # number of elements in first row
    return num_rows, num_cols

assert shape([[1, 2, 3], [4, 5, 6]]) == (2, 3) # 2 rows, 3 columns

If a matrix has *n* rows and *k* columns, we will refer to it as an *n x k matrix*. We can think of each row and an *n x k* matrix as a vector of length *k*, and each column as a vector of length *n*:

In [22]:
def get_row(A: Matrix, i: int) -> Vector:
    """Returns the i-th row of A (as a Vector)"""
    return A[i] # A[i] is already the ith row

def get_column(A: Matrix, i: int) -> Vector:
    """Returns the j-th column of A (as a Vector)"""
    return [A_i[j] # jth element of row A_i
            for A_i in A] # for each row A_i
    

We'll also want to be able to create a matrix given its shape and a function for generating its elements. For this we use a nested list comprehension:

In [23]:
from typing import Callable

def make_matrix(num_rows: int,
                num_cols: int,
                entry_fn: Callable[[int, int], float]) -> Matrix:
    """
    Returns a num_rows x num_cols matrix
    whose (i,j)-th entry is entry_fn(i, j)
    """
    return [[entry_fn(i, j) # given i, create a list
             for j in range(num_cols)] # [entry_fn(i, 0), ...]
            for i in range(num_rows)] # create one list for each i

With this function, we can makea 5 x 5 identity matrix (with 1s on the diagonal and 0s elsewhere):

In [24]:
def identity_matrix(n: int) -> Matrix:
    """Returns the n x n identity matrix"""
    return make_matrix(n, n, lambda i, j: 1 if i == j else 0)

assert identity_matrix(5) == [[1, 0, 0, 0, 0],
                              [0, 1, 0, 0, 0],
                              [0, 0, 1, 0, 0],
                              [0, 0, 0, 1, 0],
                              [0, 0, 0, 0, 1]]

We have several important uses for matrices:

  1. We can use a matrix to represent a dataset consisting of multiple vectors,
   simply by considering each vector as a row of the matrix.
  2. We can use an *n x k* matrix to represent a linear function that maps *k*-dimensional
  vectors to *n*-dimensional vectors
  3. Matrices can be used to represent binary relationships

## Warnings

Using lists as vectors is great for exposition but terrible for performance.

In production code, you would want to use the NumPy library, which includes a high-performance array class with all sorts of arithmetic operations included.

All of the machinery we built in this chapter is present in NumPy